In [4]:
#import
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model, preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import collections as cls
from sklearn.metrics import f1_score,make_scorer
train = pd.read_csv('train.csv')

# Preprocessing

In [5]:
## check the distribution of the y
print(np.sum(train['LEAVE'] == 1)/np.sum(train['LEAVE'] == 0))

0.9767186470459038


In [6]:
## check the distribution of the categorical variables
for c in train[['COLLEGE','REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']].columns:
               print("-----------{}---------".format(c))
               print(round(train[c].value_counts()/len(train)*100,4))
train.head(10)

-----------COLLEGE---------
one     50.4333
zero    49.5667
Name: COLLEGE, dtype: float64
-----------REPORTED_SATISFACTION---------
very_unsat    39.7722
very_sat      25.2111
unsat         19.8278
avg           10.0444
sat            5.1444
Name: REPORTED_SATISFACTION, dtype: float64
-----------REPORTED_USAGE_LEVEL---------
little         39.3444
very_high      25.5333
very_little    20.1111
high           10.0500
avg             4.9611
Name: REPORTED_USAGE_LEVEL, dtype: float64
-----------CONSIDERING_CHANGE_OF_PLAN---------
considering                 39.9278
actively_looking_into_it    24.8333
no                          20.2333
never_thought                9.8167
perhaps                      5.1889
Name: CONSIDERING_CHANGE_OF_PLAN, dtype: float64


,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,LEAVE
0,zero,28987,191,20,175953,217,28,5,unsat,very_little,considering,1
1,zero,45201,0,0,841177,160,1,15,unsat,avg,actively_looking_into_it,0
2,one,110663,0,0,902611,529,1,13,very_unsat,high,perhaps,0
3,zero,40646,169,71,772903,146,24,2,very_unsat,little,considering,1
4,one,132530,0,10,196535,559,0,6,very_unsat,avg,perhaps,0
5,zero,69391,48,18,431255,290,3,4,very_unsat,very_high,never_thought,0
6,one,27400,198,0,277793,334,29,10,unsat,very_little,no,1
7,one,149672,29,7,315684,708,4,4,very_unsat,very_high,no,1
8,one,84477,0,10,649511,260,1,5,very_unsat,high,perhaps,0
9,one,157543,209,10,272756,862,3,5,unsat,very_high,perhaps,1


In [7]:
# One-hot encoding
lb_make = LabelEncoder()
train["REPORTED_USAGE_LEVEL"] = lb_make.fit_transform(train["REPORTED_USAGE_LEVEL"])
train["CONSIDERING_CHANGE_OF_PLAN"] = lb_make.fit_transform(train["CONSIDERING_CHANGE_OF_PLAN"])
train["COLLEGE"] = lb_make.fit_transform(train["COLLEGE"])
train["REPORTED_SATISFACTION"] = lb_make.fit_transform(train["REPORTED_SATISFACTION"])

In [8]:
#fill NA
train = train.fillna(train.mean())

# Feature Selection

In [9]:
#PCA
pca = PCA(n_components=6)
xnames = train.columns.values.tolist()[:11]

Xtr = np.array(train[xnames])
ytr = np.array(train[['LEAVE']])

# Xtrs = preprocessing.scale(Xtr)

Xtrs = pca.fit_transform(Xtr)
print(Xtrs.shape)

pca.explained_variance_ratio_

(18000, 6)


array([9.73456794e-01, 2.65427501e-02, 3.31121575e-07, 1.13225138e-07,
       1.11340688e-08, 4.88914248e-10])

# Estimator(Use Logistic Regression )

In [10]:
# CROSS VALIDATION
from sklearn.model_selection import KFold
from sklearn import metrics
nfold = 10
kf = KFold(n_splits=nfold, shuffle=True)
auc = []
for train, test in kf.split(Xtrs):
    Xtr_split = Xtrs[train,:]
    ytr_split = ytr[train]
    Xts_split = Xtrs[test,:]
    yts_split = ytr[test]
    logreg = linear_model.LogisticRegression(C=1e5)
    logreg.fit(Xtr_split,ytr_split)
    yhat = logreg.predict(Xts_split)
    yprob = logreg.predict_proba(Xts_split)
    auci = metrics.roc_auc_score(yts_split, yprob[:, 1])
    auc.append(auci)
aucm = np.mean(auc)
print('auc = {0:.4f}'.format(aucm))

auc = 0.6955


# Estimator(Use Random Forest )

In [35]:
import sklearn.ensemble as kn
rf=kn.RandomForestClassifier()
tree_num=[50,100,150,200,250,300,350,400,450,500,550,600]
max_depth=[7,8,9,10,11,12,13]
parameters ={'n_estimators':tree_num,'max_depth':max_depth}
f1 = make_scorer(f1_score, average='micro')
clf = GridSearchCV(rf,parameters,cv=5,scoring=f1)
clf.fit(Xtrs,ytr.ravel())

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600], 'max_depth': [7, 8, 9, 10, 11, 12, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0)

In [36]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
clf.best_params_

{'max_depth': 8, 'n_estimators': 600}

In [38]:
clf.best_score_

0.7047222222222222

In [42]:
best_model = kn.RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [44]:
from sklearn import metrics
best_model.fit(Xtrs,ytr.ravel())
yhat = best_model.predict(Xtrs)
print(np.mean(yhat == ytr))
yprob = best_model.predict_proba(Xtrs)
auc = metrics.roc_auc_score(ytr, yprob[:, 1])
print(auc)

0.4998612839506173
0.8316319779864773


# Estimator(Use XGBClassifier )

In [ ]:
Procedure to adjust the parameters of xgboost


# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'n_estimators':[85,88,90,93,95]}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2),}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05,n_estimators=90),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'max_depth':[6,7,8], 'min_child_weight':[4,5,6]}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05, n_estimators=90),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'gamma':[i/10.0 for i in range(0,5)]}
# gsearch = GridSearchCV(XGBClassifier(learing_rate=0.05,max_depth=6, min_child_weight=4, n_estimators=90),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'subsample':[i/10.0 for i in range(6,10)], 'colsample_bytree':[i/10.0 for i in range(6,10)]}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05,max_depth=6, min_child_weight= 4, n_estimators=90,gamma=0.2),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'subsample':[i/100.0 for i in range(75,100,5)], 'colsample_bytree':[i/100.0 for i in range(70,95,5)]}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05,max_depth=6, min_child_weight= 4, n_estimators=90,gamma=0.2),parameters,cv=5,scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)

# xtr,xts,ytr,yts = train_test_split(Xtrs, ytr, test_size=0.33)
# parameters ={'reg_alpha':[0.038,0.04,0.042,0.044,0.046]}
# gsearch = GridSearchCV(XGBClassifier(learning_rate=0.05,max_depth=6, min_child_weight=4, n_estimators=90,gamma=0,colsample_bytree=0.85,subsample=0.85), parameters, cv=5, scoring='roc_auc')
# gsearch.fit(xtr,np.array(ytr).ravel())
# print(gsearch.best_params_)



In [11]:
# CROSS VALIDATION
from sklearn.model_selection import KFold
from sklearn import metrics
nfold = 5
kf = KFold(n_splits=nfold, shuffle=True)
auc = []
para_mcw = []
for train, test in kf.split(Xtrs):
    Xtr_split = Xtrs[train,:]
    ytr_split = ytr[train]
    Xts_split = Xtrs[test,:]
    yts_split = ytr[test]
    #best param :learning_rate=0.05,max_depth=6, min_child_weight=4, n_estimators=90,gamma=0,colsample_bytree=0.85,subsample=0.85
    model = XGBClassifier(learning_rate=0.05,max_depth=6, min_child_weight=4, n_estimators=90,gamma=0,colsample_bytree=0.85,subsample=0.85)
    model.fit(Xtr_split, np.array(ytr_split).ravel())
    yprob = model.predict_proba(Xts_split)
    auci = metrics.roc_auc_score(yts_split, yprob[:, 1])
    auc.append(auci)
aucm = np.mean(auc)
print('auc = {0:.4f}'.format(aucm))

auc = 0.7727
